# Instalação de Pacotes

In [14]:
import ipeadatapy as ipd
import pandas as pd
from matplotlib import pyplot as plt
import datetime as datetime
import numpy as np
import itertools
import time

# Coleta de Dados

### Coleta de dados de frequência Mensal.

In [ ]:
metadados_ipeamensal = ipd.metadata(frequency="Mensal")
codes_ipeamensal = ipd.metadata(frequency="Mensal")["CODE"]
data = pd.DataFrame()
for i in codes_ipeamensal:
    serie = ipd.timeseries(series=i)
    data = data._append(serie)

### Desconsiderar medidas

In [4]:
codes_filtered = []
metadados_filtered = pd.DataFrame()

measures = ['Pence', 'Reis', 'Marco alemao', 'Peso argentino', 'Franco belga', 'Peso chileno', 'Yuan', 'Peso colombiano', 'Won sul-coreano', 'Peseta espanhola', 'Franco frances', 'Florim holandes', 'Lira italiana', 'Iene japones', 'Peso novo mexicano', 'Guarani paraguaio', 'Libra esterlina', 'Peso uruguaio', 'Bolivar venezuelano', 'R$ de 2010', 'Cz$']

for index, dado in metadados_ipeamensal.iterrows():
    if(dado['MEASURE'] not in measures):
        codes_filtered.append(dado['CODE'])
        metadados_filtered = metadados_filtered._append(dado)

In [ ]:
data = pd.DataFrame()
for i in codes_filtered:
    serie = ipd.timeseries(series=i)
    data = data._append(serie)

### Organização do dataset

In [ ]:
n_data = pd.DataFrame(columns=['CODE', 'RAW DATE', 'MEDIDA','MONTH', 'YEAR', 'VALUE', 'NIVNOME', 'LAST_UPDATE'])

for i, j in data.iterrows():
    medida = metadados_ipeamensal.loc[metadados_ipeamensal['CODE'] == j['CODE']]['MEASURE'][0]
    ultima_atualizacao = metadados_ipeamensal.loc[metadados_ipeamensal['CODE'] == j['CODE']]['LAST UPDATE'][0]
    valor = str('VALUE (' + medida + ')')
    n_data = n_data._append({
                'CODE': j['CODE'],
                'RAW DATE': j['RAW DATE'],
                'MEDIDA': medida,
                'MONTH': j['MONTH'],
                'YEAR': j['YEAR'],
                'VALUE': j[valor],
                'NIVNOME': j['NIVNOME'],
                'LAST_UPDATE': ultima_atualizacao
            }, ignore_index=True)

/tmp/ipykernel_5974/1348013110.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  medida = metadados_ipeamensal.loc[metadados_ipeamensal['CODE'] == j['CODE']]['MEASURE'][0]
/tmp/ipykernel_5974/1348013110.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ultima_atualizacao = metadados_ipeamensal.loc[metadados_ipeamensal['CODE'] == j['CODE']]['LAST UPDATE'][0]
/tmp/ipykernel_5974/1348013110.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the o

### Seleção de indicadores com abrangência nacional

In [ ]:
select * from n_data
where (NIVNOME = 'Brasil' or NIVNOME is null)
and VALUE is not null;

### Seleção de séries que possuem dados de 2000 a 2020

In [ ]:
--Filtrar as series que foram atualizadas depois de 2000.
select * from n_data where "LAST UPDATE" > '2000/01/01';

--Selecionar valores apenas entre 2000 e 2020
SELECT * FROM n_data WHERE YEAR BETWEEN 2000 AND 2020;

--Agrupa as series por codigo
select code, count(*) as cont from n_data GROUP BY code;

--Seleciona apenas as series que possuem uma quantidade de dados maior ou igual a 252.
select * from serie_mensal_group where cont >= 252;

### Retirar valores nulos e duplicados da base de dados

In [12]:
#Retirar valores nulos da base de dados
data = n_data.dropna(subset=['VALUE'])

#Retirar valores duplicados da base de dados
data = n_data.drop_duplicates()

### Correção mensal e padronização de tipo

In [17]:
import datetime

for i,j in data.iterrows():
        input_date = j['RAW DATE']
        parsed_date = datetime.datetime.strptime(
            input_date, "%Y-%m-%dT%H:%M:%S%z"
        )
        formatted_date = parsed_date.strftime("%m")
        data.loc[i, 'MONTH'] = formatted_date;

data['VALUE'] = data['VALUE'].astype(float)

### Correção do IGP-M

In [ ]:
igpm_mensal = pd.read_csv('igpmensal.csv')
igpm_mensal = igpm_mensal.dropna(axis=1)
igpm_mensal =  list(igpm_mensal["IGP-M"])

igpm = map(float,igpm_mensal)
igpm = list(map(lambda x: 1+(x/100),igpm_mensal))

filterRS_data = data.query('MEDIDA == "R$" or MEDIDA == "R$ (do último mês)"')

codeRSList = set(list(data['CODE'].values))

for code in codeRSList:
    month=0
    for data_i in data.query("CODE == @code").iterrows():
        for igpm_i in range(month, 252):
            data.loc[data_i[0], "VALUE"] = (
                data.loc[data_i[0], "VALUE"]*(igpm[igpm_i])
            )
        month += 1

### Exportação dos dados

In [20]:
data.to_csv('dataset_ipeadata.csv', index=False)
metadados_filtered.to_csv('metadados_ipeadata.csv', index=False)

### Organização de códigos para cálculo de correlação

In [ ]:
select DISTINCT(code) from data;

In [ ]:
comb = list(itertools.combinations(df, 2))

corr_mensal_ = pd.DataFrame(comb, columns =['CODE1', 'CODE2'])

comb_delay = list(itertools.combinations(df['CODE'].unique(), 2))

delay_range = list(range(1, 13))

corr_delay_ = pd.DataFrame(
    [(c1, c2, delay) for c1, c2 in comb_delay for delay in delay_range], 
    columns=['CODE1', 'CODE2', 'DELAY']
)

### Cálculo de correlação

In [59]:
for idx, codeName in corr_mensal_.iterrows():
    codeName1 = codeName["CODE1"]
    codeName2 = codeName["CODE2"]
    dataCode1 = data.query("CODE == @codeName1").reset_index()
    dataCode2 = data.query("CODE == @codeName2").reset_index()
    
    dataFrame = pd.DataFrame({'CODE1': dataCode1["VALUE"], 'CODE2': dataCode2["VALUE"]})
    corrrelation = dataFrame["CODE1"].corr(dataFrame["CODE2"])

    corr_mensal_.loc[idx, "CORRELATION"] = corrrelation

### Cálculo de correlação (delay)

In [ ]:
def delay_correlation(data1, data2, delay):
    n = len(data1)
    correlation = np.corrcoef(data1[:n-delay], data2[delay:])[0, 1]
    return correlation

for idx, row in corr_delay_.iterrows():
    codeName1 = row["CODE1"]
    codeName2 = row["CODE2"]
    delay = row["DELAY"]
    dataCode1 = data.query("CODE == @codeName1").reset_index()
    dataCode2 = data.query("CODE == @codeName2").reset_index()
    
    correlation = delay_correlation(dataCode1["VALUE"], dataCode2["VALUE"], delay)
        
    corr_delay_.loc[idx, 'CORRELATION'] = correlation

In [60]:
corr_mensal_.to_csv('correlacao_mensal.csv', index=False)
corr_delay_.to_csv('correlacao_delay.csv', index=False)